# WSI to WSD: соотнесение выделенных кластеров со словарными определениями 

## 1. Часть датасета RUSSE 

In [1]:
import pandas

In [ ]:
# перенести сюда часть с формированием этого датасета (из RUSSE_ALL)

In [2]:
merged = pandas.read_csv("C:\\Users\\boss\\Documents\\Diploma\\russe-wsi-kit\\preprocessed\\merged_df.csv", 
                         "\t", encoding="utf-8")

In [3]:
merged.shape

(3160, 8)

In [4]:
# это датафрейм, в котором находятся слова и контексты, с которыми мы работаем
merged.head()

,context_id,word,gold_sense_id,predict_sense_id,positions,context,lem_context,source
0,1113,лавка,1,NaN,71-75,"забегал перед ней, засуетился, потом упал на к...","забегать перед она , засуетиться , потом упаст...",bts_rnc
1,1114,лавка,1,NaN,72-76,весь матч Знарок стоит на скамейке запасных пе...,весь матч знарка стоить на скамейка запасный п...,bts_rnc
2,1115,лавка,1,NaN,68-73,просто не умеют разыгрывать лишнего. Они же в ...,просто не уметь разыгрывать лишний . они же в ...,bts_rnc
3,1116,лавка,1,NaN,68-72,"те сдохнуть! Ин подь сюда! Вывел её на улицу, ...",тот сдохнуть ! ин подь сюда ! вывести её на ул...,bts_rnc
4,1117,лавка,1,NaN,85-89,жилищ соответствовала и вся прочая обстановка:...,жилища соответствовать и весь прочий обстановк...,bts_rnc


In [5]:
words_dict = dict()

for word in merged["word"]:
    if word not in words_dict:
        words_dict.setdefault(word, set())

for target_word in words_dict:
    for sense_id in merged[merged.word == target_word]["gold_sense_id"]:
        words_dict[target_word].add(sense_id)

In [6]:
# преобразуем set'ы в листы, чтобы был определнённый порядок, и сортируем по возрастанию
for word in words_dict:
    words_dict[word] = sorted(list(words_dict[word]))

In [7]:
# какие значения у каждого слова
words_dict

{'лавка': [1, 2],
 'замок': [1, 2],
 'винт': [1, 2],
 'лук': [1, 2],
 'суда': [1, 2],
 'дисциплина': [1, 2],
 'корона': [1, 2],
 'кран': [1, 2],
 'летопись': [1, 2],
 'балка': [1, 2],
 'вид': [1, 5],
 'горн': [1, 3],
 'жаба': [1, 4],
 'клетка': [1, 4],
 'курица': [1, 2],
 'мишень': [1, 2],
 'пост': [1, 2],
 'проказа': [1, 2],
 'пропасть': [1, 2],
 'пытка': [1, 2],
 'рысь': [1, 2],
 'среда': [2, 3, 4],
 'штамп': [1, 4]}

In [8]:
# посмотрим, сколько контекстов на каждое значение
for word in words_dict:
    print(word, [len(merged[merged.word == word][merged.gold_sense_id == gold]) for gold in words_dict[word]])

лавка [92, 147]
замок [146, 83]
винт [253, 264]
лук [65, 45]
суда [100, 35]
дисциплина [30, 64]
корона [48, 23]
кран [33, 61]
летопись [70, 25]
балка [81, 38]
вид [38, 36]
горн [20, 30]
жаба [79, 27]
клетка [38, 96]
курица [62, 31]
мишень [88, 33]
пост [32, 98]
проказа [95, 51]
пропасть [92, 25]
пытка [116, 27]
рысь [82, 38]
среда [32, 47, 52]
штамп [45, 47]


C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


## 2. Датасеты с некоторым количеством ошибок 

In [13]:
import numpy as np

In [14]:
def create_errors_df(df, percent):
    """
    Возвращает датафрейм, в конец которого для каждого значения добавлены неправильные контексты - 
    контексты из других значений этого слова, у которых gold_sense_id заменено.
    Добавляем столько, чтобы в получившемся датафрейме содержался заданный процент ошибок.
    """
    
    df_to_return = df
    for word in words_dict:
        
        # количество контекстов для каждого из значений, напр [92, 25]
        list_with_num_of_contexts = [len(df[df.word == word][df.gold_sense_id == gold]) for gold in words_dict[word]] 
        
        # сколько неправильных контекстов (из других значений) нам нужно добавить, чтобы был необходимый процент ошибки
        list_with_num_of_contexts_from_other = []
        for num in list_with_num_of_contexts:
            list_with_num_of_contexts_from_other.append(round( (num/(1-percent)) - num ))
        
        # теперь в конец df_to_return будем для каждого из значений добавлять опр количество 
        # рандомных строк из других значений, заменяя там gold_sense_id
        for i, sense in enumerate(words_dict[word]):
            num_of_new = list_with_num_of_contexts_from_other[i]
            
            # если нужно добавить больше контекстов, чем есть в другом (других) значениях
            if num_of_new > sum(list_with_num_of_contexts) - list_with_num_of_contexts[i]:
                # тогда сначала смотрим, насколько больше (в разы)
                how_many_times = num_of_new // ( sum(list_with_num_of_contexts) - list_with_num_of_contexts[i] )
                # выведем слова, для которых разница слишком большая
                if how_many_times >= 2:
                    print(word)
                
                # датафрейм с контекстами всех остальных значений
                other_contexts = df[df.word == word].drop(df[df.word == word][df.gold_sense_id == sense].index) 
                other_contexts["gold_sense_id"] = np.array(sense)
                
                # теперь добавим other_contexts в конец нашего датафрейма how_many_times раз
                for k in range(how_many_times):
                    df_to_return = pandas.concat((df_to_return, other_contexts))
                
                # сколько ещё осталось добавить
                num_of_new = num_of_new - other_contexts.shape[0] * how_many_times

                # вот теперь берём рандомные
                random = other_contexts.sample(num_of_new)
                df_to_return = pandas.concat((df_to_return, random))
                
            else:
                other_contexts = df[df.word == word].drop(df[df.word == word][df.gold_sense_id == sense].index)
                # теперь отсюда нужно выбрать num_of_new рандомных строк, заменить в них gold_sense_id на sesne
                # и добавить полученный датафрейм в конец df
                random = other_contexts.sample(num_of_new)
                random["gold_sense_id"] = np.array(sense)
                df_to_return = pandas.concat((df_to_return, random))
        
    return df_to_return

In [15]:
df_with_10_errors = create_errors_df(merged, 0.1)
df_with_10_errors.shape

C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':
C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


(3511, 8)

In [16]:
df_with_20_errors = create_errors_df(merged, 0.2)
df_with_30_errors = create_errors_df(merged, 0.3)
df_with_40_errors = create_errors_df(merged, 0.4)
df_with_40_errors.shape # на 40% проверяли, правильно работает - йеп
# выводятся слова, для которых разница между количеством контекстов в значениях слишком большая (больше, чем в два раза)

C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':
C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


пост
пропасть
пытка


(5264, 8)

In [17]:
path_to_write = "C:\\Users\\boss\\Documents\\Diploma\\russe-wsi-kit\\preprocessed\\"

In [ ]:
# в новых датафреймах индексы идут не подряд, а некотрые повторяются. Чтобы от этого избавиться, запишем в csv и считаем заново
df_with_10_errors.to_csv(path_to_write + "10_errors.csv", "\t", encoding="utf-8", index=False)
df_with_20_errors.to_csv(path_to_write + "20_errors.csv", "\t", encoding="utf-8", index=False)
df_with_30_errors.to_csv(path_to_write + "30_errors.csv", "\t", encoding="utf-8", index=False)
df_with_40_errors.to_csv(path_to_write + "40_errors.csv", "\t", encoding="utf-8", index=False)

In [18]:
df_with_10_errors = pandas.read_csv(path_to_write + "10_errors.csv", "\t", encoding="utf-8")
df_with_20_errors = pandas.read_csv(path_to_write + "20_errors.csv", "\t", encoding="utf-8")
df_with_30_errors = pandas.read_csv(path_to_write + "30_errors.csv", "\t", encoding="utf-8")
df_with_40_errors = pandas.read_csv(path_to_write + "40_errors.csv", "\t", encoding="utf-8")

## 3. В датасетах - идеальном и четырёх с ошибкой - для каждого значения выделяем ключевые слова

In [19]:
from nltk.probability import FreqDist
import collections
import codecs

In [20]:
def get_words_contexts_dict(df): # {"target_word":{1:"все лем контексты", 2:"все лем контексты"}, "target_word2": ...}
    dict_to_return = dict()
    for word in words_dict:
        senses = words_dict[word]
        dict_to_return[word] = dict()
        for sense in senses:
            dict_to_return[word][sense] = "\n".join(list(df[df.word == word][df.gold_sense_id == sense]["lem_context"]))
    return dict_to_return

In [21]:
perfect_df = get_words_contexts_dict(merged)
errors_10 = get_words_contexts_dict(df_with_10_errors)
errors_20 = get_words_contexts_dict(df_with_20_errors)
errors_30 = get_words_contexts_dict(df_with_30_errors)
errors_40 = get_words_contexts_dict(df_with_40_errors)

C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [22]:
def get_freq_dist(words_contexts_dict): # {"target_word":{1:распределение частот слов, 2:распределение}, "target_word2": ...}
    dist_to_return = dict()
    for word in words_contexts_dict:
        dist_to_return[word] = dict()
        for sense in words_contexts_dict[word]:
            dist_to_return[word][sense] = FreqDist(words_contexts_dict[word][sense].split())
    return dist_to_return

In [23]:
fdist_perf = get_freq_dist(perfect_df)
fdist_10 = get_freq_dist(errors_10)
fdist_20 = get_freq_dist(errors_20)
fdist_30 = get_freq_dist(errors_30)
fdist_40 = get_freq_dist(errors_40)

In [24]:
stw_file = "C:\\Users\\boss\\Documents\\Diploma\\my_stopwords.txt"
with codecs.open(stw_file, encoding = 'utf-8') as f:
    stw_set = set(f.read().split())

In [25]:
def get_dist_wo_stopwords(fdist_dict): # создадим словарь с распределением частот слов, но без стоп-слов
    dist_to_return = dict()
    for target_word in fdist_dict:
        dist_to_return[target_word] = dict()
        for sense in fdist_dict[target_word]:
            dist_to_return[target_word][sense] = dict()
            fdist_for_sense = fdist_dict[target_word][sense]
            for word in fdist_for_sense:
                if word in stw_set: # если слово в списке стоп слов
                    pass
                elif True in set(char in set(list("1234567890")) for char in word): # если в слове содержится цифра
                    pass
                elif True in set(char in set(list("abcdefghijklmnopqrstuvwxyz")) for char in word): # если латинские буквы
                    pass
                else:
                    dist_to_return[target_word][sense][word] = fdist_for_sense[word]
    return dist_to_return

In [26]:
fdist_perf_wo_stopwords = get_dist_wo_stopwords(fdist_perf)
fdist_10_wo_stopwords = get_dist_wo_stopwords(fdist_10)
fdist_20_wo_stopwords = get_dist_wo_stopwords(fdist_20)
fdist_30_wo_stopwords = get_dist_wo_stopwords(fdist_30)
fdist_40_wo_stopwords = get_dist_wo_stopwords(fdist_40)

In [255]:
def get_most_frequent_words_per_sense(dist_wo_stopwords, n=20): # словарь, в котором для каждого значения n самых частотных слов
    dict_to_return = dict()

    for target_word in dist_wo_stopwords:
        dict_to_return[target_word] = dict()
        for sense in dist_wo_stopwords[target_word]: 
            dict_per_sense = dist_wo_stopwords[target_word][sense]
            sorted_dict_per_sense = collections.OrderedDict(sorted(dict_per_sense.items(), key=lambda kv: kv[1], 
                                                                   reverse=True)[:n])
            dict_to_return[target_word][sense] = sorted_dict_per_sense
    return dict_to_return

In [28]:
most_frequent_perfect = get_most_frequent_words_per_sense(fdist_perf_wo_stopwords)
most_frequent_10_errors = get_most_frequent_words_per_sense(fdist_10_wo_stopwords)
most_frequent_20_errors = get_most_frequent_words_per_sense(fdist_20_wo_stopwords)
most_frequent_30_errors = get_most_frequent_words_per_sense(fdist_30_wo_stopwords)
most_frequent_40_errors = get_most_frequent_words_per_sense(fdist_40_wo_stopwords)

In [238]:
most_frequent_10_errors["дисциплина"]

{1: OrderedDict([('дисциплина', 33),
              ('процесс', 5),
              ('необходимый', 5),
              ('год', 5),
              ('волос', 5),
              ('жизнь', 4),
              ('должный', 4),
              ('учебный', 3),
              ('правонарушение', 3),
              ('работа', 3),
              ('служебный', 3),
              ('строгий', 3),
              ('человек', 3),
              ('труд', 3),
              ('трудовой', 3),
              ('инструкция', 3),
              ('нарушение', 2),
              ('особенность', 2),
              ('экономика', 2),
              ('уровень', 2)]),
 2: OrderedDict([('дисциплина', 77),
              ('учебный', 9),
              ('работа', 8),
              ('изучение', 7),
              ('математика', 7),
              ('год', 7),
              ('специальность', 6),
              ('программа', 6),
              ('преподаватель', 6),
              ('наука', 6),
              ('научный', 6),
              ('кафедра', 5),


In [256]:
ten_most_frequent_perfect = get_most_frequent_words_per_sense(fdist_perf_wo_stopwords, n=10)
ten_most_frequent_10_errors = get_most_frequent_words_per_sense(fdist_10_wo_stopwords, n=10)
ten_most_frequent_20_errors = get_most_frequent_words_per_sense(fdist_20_wo_stopwords, n=10)
ten_most_frequent_30_errors = get_most_frequent_words_per_sense(fdist_30_wo_stopwords, n=10)
ten_most_frequent_40_errors = get_most_frequent_words_per_sense(fdist_40_wo_stopwords, n=10)

In [257]:
ten_most_frequent_10_errors["дисциплина"]

{1: OrderedDict([('дисциплина', 33),
              ('процесс', 5),
              ('необходимый', 5),
              ('год', 5),
              ('волос', 5),
              ('жизнь', 4),
              ('должный', 4),
              ('учебный', 3),
              ('правонарушение', 3),
              ('работа', 3)]),
 2: OrderedDict([('дисциплина', 77),
              ('учебный', 9),
              ('работа', 8),
              ('изучение', 7),
              ('математика', 7),
              ('год', 7),
              ('специальность', 6),
              ('программа', 6),
              ('преподаватель', 6),
              ('наука', 6)])}

In [267]:
thirty_most_frequent_perfect = get_most_frequent_words_per_sense(fdist_perf_wo_stopwords, n=30)
thirty_most_frequent_10_errors = get_most_frequent_words_per_sense(fdist_10_wo_stopwords, n=30)
thirty_most_frequent_20_errors = get_most_frequent_words_per_sense(fdist_20_wo_stopwords, n=30)
thirty_most_frequent_30_errors = get_most_frequent_words_per_sense(fdist_30_wo_stopwords, n=30)
thirty_most_frequent_40_errors = get_most_frequent_words_per_sense(fdist_40_wo_stopwords, n=30)

## 4. Из ключевых слов оставляем только хорошие по косинусу 

In [30]:
import zipfile
import gensim
from sklearn.metrics.pairwise import cosine_similarity as cos
import pymorphy2
import copy
morph = pymorphy2.MorphAnalyzer()

C:\Users\boss\Anaconda3\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [31]:
model_file = "C:\\Users\\boss\\Documents\\Diploma\\182.zip"
with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    model_skipgram = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

In [32]:
def get_dict_with_words_per_sense(dict_with_word_frequencies):
    dict_to_return = dict()
    for target_word in dict_with_word_frequencies:
        dict_to_return[target_word] = dict()
        for sense in dict_with_word_frequencies[target_word]:
            ordered_dict = dict_with_word_frequencies[target_word][sense]
            dict_to_return[target_word][sense] = [word for word in ordered_dict]
    
    return dict_to_return

In [33]:
dict_with_words_per_sense_perfect = get_dict_with_words_per_sense(most_frequent_perfect)
dict_with_words_per_sense_10_errors = get_dict_with_words_per_sense(most_frequent_10_errors)
dict_with_words_per_sense_20_errors = get_dict_with_words_per_sense(most_frequent_20_errors)
dict_with_words_per_sense_30_errors = get_dict_with_words_per_sense(most_frequent_30_errors)
dict_with_words_per_sense_40_errors = get_dict_with_words_per_sense(most_frequent_40_errors)

In [258]:
dict_with_ten_words_per_sense_perfect = get_dict_with_words_per_sense(ten_most_frequent_perfect)
dict_with_ten_words_per_sense_10_errors = get_dict_with_words_per_sense(ten_most_frequent_10_errors)
dict_with_ten_words_per_sense_20_errors = get_dict_with_words_per_sense(ten_most_frequent_20_errors)
dict_with_ten_words_per_sense_30_errors = get_dict_with_words_per_sense(ten_most_frequent_30_errors)
dict_with_ten_words_per_sense_40_errors = get_dict_with_words_per_sense(ten_most_frequent_40_errors)

In [268]:
dict_with_thirty_words_per_sense_perfect = get_dict_with_words_per_sense(thirty_most_frequent_perfect)
dict_with_thirty_words_per_sense_10_errors = get_dict_with_words_per_sense(thirty_most_frequent_10_errors)
dict_with_thirty_words_per_sense_20_errors = get_dict_with_words_per_sense(thirty_most_frequent_20_errors)
dict_with_thirty_words_per_sense_30_errors = get_dict_with_words_per_sense(thirty_most_frequent_30_errors)
dict_with_thirty_words_per_sense_40_errors = get_dict_with_words_per_sense(thirty_most_frequent_40_errors)

In [34]:
def return_pymorphy_pos(word):
    p = morph.parse(word)[0]
    if "INFN" in p.tag:
        pos = "VERB"
    elif "ADJF" in p.tag:
        pos = "ADJ"
    elif "ADVB" in p.tag:
        pos = "ADV"
    else:
        pos = str(p.tag.POS)
    return word + "_" + pos # ! не p.normal_form, потому что это уже нормализованные тексты

In [35]:
def get_dict_with_enhanced_pymorphy_pos(dict_with_words_per_sense):
    dict_to_retrun = dict()

    for target_word in dict_with_words_per_sense:
        dict_to_retrun[target_word+"_NOUN"] = dict()
        for sense in dict_with_words_per_sense[target_word]:
            list_with_pos = []
            list_of_words_without_target = dict_with_words_per_sense[target_word][sense][1:] # потому что первое слово - целевое
            for word in list_of_words_without_target:
                list_with_pos.append(return_pymorphy_pos(word))
            dict_to_retrun[target_word+"_NOUN"][sense] = list_with_pos
    return dict_to_retrun

In [38]:
pos_dict_perfect = get_dict_with_enhanced_pymorphy_pos(dict_with_words_per_sense_perfect)
pos_dict_10_errors = get_dict_with_enhanced_pymorphy_pos(dict_with_words_per_sense_10_errors)
pos_dict_20_errors = get_dict_with_enhanced_pymorphy_pos(dict_with_words_per_sense_20_errors)
pos_dict_30_errors = get_dict_with_enhanced_pymorphy_pos(dict_with_words_per_sense_30_errors)
pos_dict_40_errors = get_dict_with_enhanced_pymorphy_pos(dict_with_words_per_sense_40_errors)

In [259]:
pos_dict_ten_perfect = get_dict_with_enhanced_pymorphy_pos(dict_with_ten_words_per_sense_perfect)
pos_dict_ten_10_errors = get_dict_with_enhanced_pymorphy_pos(dict_with_ten_words_per_sense_10_errors)
pos_dict_ten_20_errors = get_dict_with_enhanced_pymorphy_pos(dict_with_ten_words_per_sense_20_errors)
pos_dict_ten_30_errors = get_dict_with_enhanced_pymorphy_pos(dict_with_ten_words_per_sense_30_errors)
pos_dict_ten_40_errors = get_dict_with_enhanced_pymorphy_pos(dict_with_ten_words_per_sense_40_errors)

In [269]:
pos_dict_thirty_perfect = get_dict_with_enhanced_pymorphy_pos(dict_with_thirty_words_per_sense_perfect)
pos_dict_thirty_10_errors = get_dict_with_enhanced_pymorphy_pos(dict_with_thirty_words_per_sense_10_errors)
pos_dict_thirty_20_errors = get_dict_with_enhanced_pymorphy_pos(dict_with_thirty_words_per_sense_20_errors)
pos_dict_thirty_30_errors = get_dict_with_enhanced_pymorphy_pos(dict_with_thirty_words_per_sense_30_errors)
pos_dict_thirty_40_errors = get_dict_with_enhanced_pymorphy_pos(dict_with_thirty_words_per_sense_40_errors)

In [96]:
class GoodWord(object):
    """
    Класс для набора хороших ключевых слов для определённого значения.
    Хорошие - у которых косинус с целевым словом (self.cosine) больше определённого порога. Также тут хранится 
    найденный для  них w2v вектор, который потом будет нужен для вычисления среднего вектора хороших слов.
    """
    def __init__(self, lemma, pos_word, vec, cosine):
        self.lemma = lemma
        self.pos_word = pos_word
        self.vec = vec
        self.cosine = cosine
    
    def __repr__(self):
        return self.lemma

In [97]:
def get_dict_with_good_words(dict_with_enhanced_pymorphy_pos, threshold=0.3):

    # важно, что в UDPipe и основанной на ней модели W2V вроде бы нет слов с "ё", а в ESA они с "ё", поэтому здесь
    # для получения векторов делаем replace, но сами слова оставляем на будущее с "ё"
    dict_to_return = dict()

    for target_word in dict_with_enhanced_pymorphy_pos:
        dict_to_return[target_word] = dict()
        try:
            target_vector = model_skipgram[target_word].reshape(1,300)
        except KeyError: # такого слова нет в словаре. Так как мы просто добавляли к target_word тег NOUN, теперь можем 
            # попробовать нормализовать слово
            normalized = morph.parse(target_word[:target_word.index("_")])[0].normal_form
            target_vector = model_skipgram[normalized.replace("ё", "е") + "_NOUN"].reshape(1,300)
            
        for sense in dict_with_enhanced_pymorphy_pos[target_word]:
            good_list = []
            for word in dict_with_enhanced_pymorphy_pos[target_word][sense]:
                try:
                    word_vector = model_skipgram[word.replace("ё", "е")].reshape(1,300)
                    c = cos(target_vector,word_vector).tolist()[0][0]
                except KeyError:
                    c = 0
                    # print(word) - выводить те, которых не оказалось в модели W2V
                if c >= threshold:
                    good_list.append(GoodWord(lemma=word[:word.index("_")], pos_word=word, 
                                              vec=word_vector, cosine=round(c, 3)))
                    
            dict_to_return[target_word][sense] = good_list
    return dict_to_return

In [98]:
good_words_from_perfect = get_dict_with_good_words(pos_dict_perfect) # выводятся те, которых нет в модели W2V
good_words_from_10_errors = get_dict_with_good_words(pos_dict_ten_10_errors) 
good_words_from_20_errors = get_dict_with_good_words(pos_dict_ten_20_errors) 
good_words_from_30_errors = get_dict_with_good_words(pos_dict_ten_30_errors) 
good_words_from_40_errors = get_dict_with_good_words(pos_dict_ten_40_errors) 

In [260]:
ten_good_words_from_perfect = get_dict_with_good_words(pos_dict_ten_perfect) 
ten_good_words_from_10_errors = get_dict_with_good_words(pos_dict_ten_10_errors) 
ten_good_words_from_20_errors = get_dict_with_good_words(pos_dict_ten_20_errors) 
ten_good_words_from_30_errors = get_dict_with_good_words(pos_dict_ten_30_errors) 
ten_good_words_from_40_errors = get_dict_with_good_words(pos_dict_ten_40_errors) 

In [270]:
thirty_good_words_from_perfect = get_dict_with_good_words(pos_dict_thirty_perfect) 
thirty_good_words_from_10_errors = get_dict_with_good_words(pos_dict_thirty_10_errors) 
thirty_good_words_from_20_errors = get_dict_with_good_words(pos_dict_thirty_20_errors) 
thirty_good_words_from_30_errors = get_dict_with_good_words(pos_dict_thirty_30_errors) 
thirty_good_words_from_40_errors = get_dict_with_good_words(pos_dict_thirty_40_errors) 

In [104]:
def sort_dict_with_good_words(dict_with_good_words): # сортируем попавшие в него хорошие слова по убыванию косинуса
    dict_to_return = copy.deepcopy(dict_with_good_words) # нужна глубокая копия, иначе dict_with_good_words сам изменяется
    for target_word in dict_to_return:
        for sense in dict_to_return[target_word]:
            list_for_sense = dict_to_return[target_word][sense]
            sorted_list_for_sense = sorted(list_for_sense, key=lambda element: element.cosine, reverse = True)
            dict_to_return[target_word][sense] = sorted_list_for_sense
    return dict_to_return

In [178]:
sorted_good_words_from_perfect = sort_dict_with_good_words(good_words_from_perfect)
sorted_good_words_from_10_errors = sort_dict_with_good_words(good_words_from_10_errors)
sorted_good_words_from_20_errors = sort_dict_with_good_words(good_words_from_20_errors)
sorted_good_words_from_30_errors = sort_dict_with_good_words(good_words_from_30_errors)
sorted_good_words_from_40_errors = sort_dict_with_good_words(good_words_from_40_errors)

In [261]:
sorted_ten_good_words_from_perfect = sort_dict_with_good_words(ten_good_words_from_perfect)
sorted_ten_good_words_from_10_errors = sort_dict_with_good_words(ten_good_words_from_10_errors)
sorted_ten_good_words_from_20_errors = sort_dict_with_good_words(ten_good_words_from_20_errors)
sorted_ten_good_words_from_30_errors = sort_dict_with_good_words(ten_good_words_from_30_errors)
sorted_ten_good_words_from_40_errors = sort_dict_with_good_words(ten_good_words_from_40_errors)

In [262]:
sorted_ten_good_words_from_perfect

{'лавка_NOUN': {1: [стол, дверь, стена, деревянный, сидеть],
  2: [магазин, торговать, дом, улица, книжный, сувенирный]},
 'замок_NOUN': {1: [замковый, башня, стена], 2: [замковый, ключ]},
 'винт_NOUN': {1: [болт,
   гайка,
   крепление,
   отверстие,
   крышка,
   головка,
   диаметр],
  2: [лопасть, двигатель, несущий, гребной, самолёт, вертолёт, машина]},
 'лук_NOUN': {1: [стрела, тетива, стрельба, лучник, оружие],
  2: [чеснок, луковица, растение, ветвистый]},
 'суда_NOUN': {1: [арбитражный, судья, верховный], 2: [судно, корабль, флот]},
 'дисциплина_NOUN': {1: [], 2: [наука, учебный, преподаватель, специальность]},
 'корона_NOUN': {1: [щит, герб, королевский, золотой, драгоценный],
  2: [король]},
 'кран_NOUN': {1: [крановый, водопроводный, вода, двигатель, вод, починить],
  2: [подъёмный, крановый, башенный, стрела, мостовый]},
 'летопись_NOUN': {1: [история, новгородский], 2: [история]},
 'балка_NOUN': {1: [крыша, потолочный, деревянный, стена, потолок],
  2: [степь, спускаться,

In [271]:
sorted_thirty_good_words_from_perfect = sort_dict_with_good_words(thirty_good_words_from_perfect)
sorted_thirty_good_words_from_10_errors = sort_dict_with_good_words(thirty_good_words_from_10_errors)
sorted_thirty_good_words_from_20_errors = sort_dict_with_good_words(thirty_good_words_from_20_errors)
sorted_thirty_good_words_from_30_errors = sort_dict_with_good_words(thirty_good_words_from_30_errors)
sorted_thirty_good_words_from_40_errors = sort_dict_with_good_words(thirty_good_words_from_40_errors)

## 5. Создание словаря с определениями слов через подключение к онлайн-словарю

In [44]:
from urllib.parse import quote
import requests
from bs4 import BeautifulSoup
import time

In [43]:
words = list(words_dict.keys())
words

['лавка',
 'замок',
 'винт',
 'лук',
 'суда',
 'дисциплина',
 'корона',
 'кран',
 'летопись',
 'балка',
 'вид',
 'горн',
 'жаба',
 'клетка',
 'курица',
 'мишень',
 'пост',
 'проказа',
 'пропасть',
 'пытка',
 'рысь',
 'среда',
 'штамп']

In [ ]:
# senses = soup.find_all('sup') - для ожегова - количество значений
# [<sup>1</sup>, <sup>2</sup>]

In [45]:
dict_to_use = "https://gufo.me/dict/efremova/"  # "https://gufo.me/dict/ozhegov/"
roman = {"I", "V", "X"} # с чего может начинаться обозначение значения

def get_definitions(words, common_dict, look_at_lemmas_not_words=True):
    
    for word in words: # слова не обязательно лемматизированы - напр., "суда"
        
        if look_at_lemmas_not_words: # итерация по возможным леммам переданных слов, а не по самим словам!
            variants = morph.parse(word)
            set_of_lemmas = set([el.normal_form for el in variants]) # все возможные леммы для слова

            for lemma in set_of_lemmas:
                if lemma not in common_dict:
                    dict_with_senses = dict()
                    encoded_word = quote(lemma)
                    content = requests.get(dict_to_use + encoded_word).text
                    soup = BeautifulSoup(content, 'html.parser')
                    text = [el.get_text() for el in soup.find_all('p')[:-1]]
                    if text == ['Слово не найдено (в индексе и в фейловере).']:
                        print("Лемма '%s' слова '%s' не найдена в словаре" % (lemma, word))
                        continue
                    else:
                        if text[0] == "1.": # это исключения - как с "рысью", когда есть ещё один, более внешний уровень 
                            # разделения значений - нас тогда здесь будет интересовать только первый кусочек, потому что 
                            # второй - это что-то специфическое
                            print("Исключение с леммой '%s' слова '%s'" % (lemma, word))
                            text = text[1:text.index("2.")]

                        text_all = " ".join(text)
                        if ("I" in text_all) and ("II" in text_all): # если есть явно разграничиваемые значения
                            for element in text[1:]: # первый элемент здесь - просто само слово
                                el_is_key = False
                                for digit in roman:
                                    if digit == element[0]: # тогда этот элемент (строка) будет ключом словаря
                                        el_is_key = True
                                        key = element
                                        dict_with_senses[key] = []
                                        break
                                if el_is_key:
                                    continue
                                else:
                                    if not element.startswith("||"): # это самостоятельное определение, добавляем его 
                                        dict_with_senses[key].append(element)
                                    else: # это оттенок значения, мы добавляем эту строку в конец предыдущего определения
                                        # continue - если вообще игнорируем оттенки значений
                                        num_of_last_el = len(dict_with_senses[key]) - 1
                                        dict_with_senses[key][num_of_last_el] = dict_with_senses[key][num_of_last_el] + \
                                                                                " " + element
                        else:
                            try:
                                dict_with_senses[text[0]] = [] # ключом будет слово + род, типа "путь м."
                            except IndexError: # возникали какие-то ошибки, если запрашивать сразу много слов, без time.sleep()
                                print(text)
                                raise IndexError("Проблема с леммой '%s' слова '%s'" % (lemma, word))
                            for element in text[1:]:
                                if not element.startswith("||"):
                                    dict_with_senses[text[0]].append(element)
                                else:
                                    num_of_last_el = len(dict_with_senses[text[0]]) - 1
                                    dict_with_senses[text[0]][num_of_last_el] = dict_with_senses[text[0]][num_of_last_el] + \
                                                                                " " + element

                    common_dict[lemma] = dict_with_senses
                    time.sleep(0.5)  # без этого не сделать подряд все слова, возникает ошибка
                    
                    
        else: # почти всё то же самое, но смотрим на сами слова
            if word not in common_dict:
                dict_with_senses = dict()
                encoded_word = quote(word)
                content = requests.get(dict_to_use + encoded_word).text
                soup = BeautifulSoup(content, 'html.parser')
                text = [el.get_text() for el in soup.find_all('p')[:-1]] # последний элемент - "© 2019 Gufo.me"

                if text == ['Слово не найдено (в индексе и в фейловере).']:
                    print("Слово '%s' не найдено в словаре" % word)
                    continue # просто переходим к следующему слову

                if text[0] == "1.": # это исключения - как с "рысью", когда есть ещё один, более внешний уровень
                    #  разделения значений - нас тогда здесь будет интересовать только первый кусочек
                    print("Исключение со словом " + word)
                    text = text[1:text.index("2.")]

                text_all = " ".join(text)
                if ("I" in text_all) and ("II" in text_all): # если есть явно разграничиваемые значения
                    for element in text[1:]: # первый элемент здесь - просто само слово
                        el_is_key = False
                        for digit in roman:
                            if digit == element[0]: # тогда это будет ключом словаря
                                el_is_key = True
                                key = element
                                dict_with_senses[key] = []
                                break
                        if el_is_key:
                            continue
                        else:
                            if not element.startswith("||"):
                                dict_with_senses[key].append(element)
                            else: # оттенки значений, идущие после ||, присоединяем к предыдущему определению
                                num_of_last_el = len(dict_with_senses[key]) - 1
                                dict_with_senses[key][num_of_last_el] = dict_with_senses[key][num_of_last_el] + " " + element
                else:
                    try:
                        dict_with_senses[text[0]] = [] # ключом будет слово + род, типа "путь м."
                    except IndexError:
                        print(text)
                        raise IndexError("Проблема со словом " + word)
                    for element in text[1:]:
                        if not element.startswith("||"):
                            dict_with_senses[text[0]].append(element)
                        else:
                            num_of_last_el = len(dict_with_senses[text[0]]) - 1
                            dict_with_senses[text[0]][num_of_last_el] = dict_with_senses[text[0]][num_of_last_el] + \
                                                                        " " + element

                common_dict[word] = dict_with_senses
                time.sleep(0.5)  # без этого не сделать подряд все слова, нужно разбивать на две части
            
    return common_dict

In [46]:
start = time.time()
dict_with_lemmas = dict()
dict_with_lemmas = get_definitions(words, dict_with_lemmas)
print(time.time()-start)
dict_with_lemmas

Исключение с леммой 'рысь' слова 'рысь'
15.45388388633728


{'лавка': {'I ж.': ['1. Скамья в доме для сидения или лежания (обычно прикрепленная к стене).',
   '2. Скамья в саду, на улице.'],
  'II ж.': ['Небольшой магазин (обычно в сельской местности).']},
 'замокнуть': {'замокнуть сов. неперех. разг.': ['см. замокать']},
 'замок': {'I замок м.': ['1. Устройство для запирания дверей, чемоданов, ящиков мебели и т.п. ключом. || Приспособление для соединения концов ювелирных изделий (ожерелья, браслета, цепочки и т.п.).',
   '2. Способ соединения бревен, брусьев, при котором они составляют одно целое.'],
  'II замок м.': ['Клинчатый камень в вершине арки, свода (в архитектуре).'],
  'III замок м.': ['Устройство в огнестрельном оружии, обеспечивающее досылание снаряда, патрона, запирание канала ствола, производство выстрела и выбрасывание гильзы; затвор I 4.'],
  'IV замок м.': ['Укрепленное жилище — дворец и крепость — феодала.']},
 'винт': {'I м.': ['1. Стержень со спиральной нарезкой, служащий для крепления или соединения деталей, частей чего-ли

In [47]:
start = time.time()
dict_with_words = dict()
dict_with_words = get_definitions(words, dict_with_words, look_at_lemmas_not_words=False)
print(time.time()-start)
dict_with_words

Слово 'суда' не найдено в словаре
Исключение со словом рысь
13.07074761390686


{'лавка': {'I ж.': ['1. Скамья в доме для сидения или лежания (обычно прикрепленная к стене).',
   '2. Скамья в саду, на улице.'],
  'II ж.': ['Небольшой магазин (обычно в сельской местности).']},
 'замок': {'I замок м.': ['1. Устройство для запирания дверей, чемоданов, ящиков мебели и т.п. ключом. || Приспособление для соединения концов ювелирных изделий (ожерелья, браслета, цепочки и т.п.).',
   '2. Способ соединения бревен, брусьев, при котором они составляют одно целое.'],
  'II замок м.': ['Клинчатый камень в вершине арки, свода (в архитектуре).'],
  'III замок м.': ['Устройство в огнестрельном оружии, обеспечивающее досылание снаряда, патрона, запирание канала ствола, производство выстрела и выбрасывание гильзы; затвор I 4.'],
  'IV замок м.': ['Укрепленное жилище — дворец и крепость — феодала.']},
 'винт': {'I м.': ['1. Стержень со спиральной нарезкой, служащий для крепления или соединения деталей, частей чего-либо.',
   '2. Приспособление в виде лопастей, предназначенное для пр

In [48]:
dict_with_lemmas.keys()

dict_keys(['лавка', 'замокнуть', 'замок', 'винт', 'лук', 'лука', 'суд', 'судно', 'дисциплина', 'корона', 'кран', 'летопись', 'балка', 'вид', 'горн', 'жаба', 'клетка', 'курица', 'мишень', 'пост', 'проказа', 'пропасть', 'пытка', 'рысь', 'среда', 'штамп'])

In [49]:
dict_with_words.keys()

dict_keys(['лавка', 'замок', 'винт', 'лук', 'дисциплина', 'корона', 'кран', 'летопись', 'балка', 'вид', 'горн', 'жаба', 'клетка', 'курица', 'мишень', 'пост', 'проказа', 'пропасть', 'пытка', 'рысь', 'среда', 'штамп'])

## 6. Предобработка полученных определений 

In [50]:
from pymorphy2.tokenizers import simple_word_tokenize

In [51]:
class SenseDefinition(object):
    """
    Класс для определений из словаря и дальнейшей работы с ними. На вход передаём только raw строку с самим определением,
    получаем обработанные строки (два варианта) и вектор w2v для этого определения.
    """
    def __init__(self, raw_text):
        self.raw_text = raw_text
        self.normalized, self.normalized_with_pos = normalize_string(raw_text, stw_set)
        self.vec = get_w2v_vec(self.normalized_with_pos)
    
    def __repr__(self):
        return "[%s,\n%s\n%s]" % (self.raw_text, self.normalized, self.normalized_with_pos)

In [74]:
def get_w2v_vec(preprocessed_string):
    list_with_vectors = []
    for word in preprocessed_string.split():
        try:
            vec = model_skipgram[word.replace("ё", "е")].reshape(1,300)
            list_with_vectors.append(vec)
        except KeyError: # ничего не делаем, и этот вектор не считается при вычислении среднего значения
            # vec = np.zeros(300).reshape(1,300)
            print("Слова '%s' нет в модели" % word)
            continue
    if list_with_vectors != []:
        mean_vec = np.mean(list_with_vectors, axis=0)
    else:
        print("У следующего определения будет нулевой вектор: " + preprocessed_string) # у значения ['см. замокать']
        mean_vec = np.zeros(300).reshape(1,300)
    return mean_vec

In [53]:
def normalize_string(string_to_normalize, stw_set): 
    """ 
    Возвращает 1) лемматизированную строку без стоп-слов, 2) ту же строку, но с тегами ЧР как в W2V.
    
    string_to_normalize - строка raw текста
    stw_set - множество стоп-слов
    """
        
    # именно для работы со словарём: вручную убираем пометы
    string = string_to_normalize.replace(" разг.", "").replace(" простореч.", "").replace(" перен.", "").replace(" устар.", "")

    lemmatized_list_for_string = [] # лемматизируем
    for token in simple_word_tokenize(string):
        if token == "см": # pymotphy заменяет его на "сантиметр", а так мы просто убираем
            continue
        lemmatized_list_for_string.append(morph.parse(token)[0].normal_form) 

    good_lemmatized_list_for_string = [] # убираем плохие токены    
    for lemmatized_token in lemmatized_list_for_string:
        if lemmatized_token in stw_set: # если слово в списке стоп слов
            pass
        elif True in set(char in set(list("1234567890")) for char in lemmatized_token): # если в слове содержится цифра
            pass
        elif True in set(char in set(list("abcdefghijklmnopqrstuvwxyz")) for char in lemmatized_token): # если латинские буквы
            pass
        else:
            good_lemmatized_list_for_string.append(lemmatized_token)
    
    lemmas_with_pos = []
    for lemma in good_lemmatized_list_for_string:
        lemmas_with_pos.append(return_pymorphy_pos(lemma))
    
    return " ".join(good_lemmatized_list_for_string), " ".join(lemmas_with_pos)

In [75]:
# нужна глубокая копия, иначе dict_with_lemmas сам будем тоже изменяться
dict_with_definition_instances = copy.deepcopy(dict_with_lemmas)

In [76]:
for lemma in dict_with_definition_instances:
    for sense in dict_with_definition_instances[lemma]:
        list_of_strings_to_normalize = dict_with_definition_instances[lemma][sense]
        dict_with_definition_instances[lemma][sense] = [SenseDefinition(string) for string in list_of_strings_to_normalize]

Слова 'замокать_VERB' нет в модели
У следующего определения будет нулевой вектор: замокать_VERB
Слова 'том_ADJ' нет в модели
Слова 'том_ADJ' нет в модели
Слова 'клинчатый_ADJ' нет в модели
Слова 'досылание_NOUN' нет в модели
Слова 'жилища_NOUN' нет в модели
Слова 'том_ADJ' нет в модели
Слова 'русь_NOUN' нет в модели
Слова 'том_ADJ' нет в модели
Слова 'том_ADJ' нет в модели
Слова 'окружающий_ADJ' нет в модели
Слова 'том_ADJ' нет в модели
Слова 'входящий_ADJ' нет в модели
Слова 'том_ADJ' нет в модели
Слова 'предусмотренный_ADJ' нет в модели
Слова 'том_ADJ' нет в модели
Слова 'духов_NOUN' нет в модели
Слова 'том_ADJ' нет в модели
Слова 'том_ADJ' нет в модели
Слова 'том_ADJ' нет в модели
Слова 'галопом_ADV' нет в модели
Слова 'шагом_ADV' нет в модели
Слова 'социально-бытовой_ADJ' нет в модели
Слова 'третье_NOUN' нет в модели
Слова 'том_ADJ' нет в модели
Слова 'путём_ADV' нет в модели
Слова 'том_ADJ' нет в модели


In [239]:
dict_with_definition_instances["дисциплина"]

{'I ж.': [[1. Определенный порядок поведения людей, отвечающий сложившимся в обществе нормам права и морали, а также требованиям той или иной организации.,
  определённый порядок поведение человек отвечать сложиться общество норма право мораль требование организация
  определённый_ADJ порядок_NOUN поведение_NOUN человек_NOUN отвечать_VERB сложиться_VERB общество_NOUN норма_NOUN право_NOUN мораль_NOUN требование_NOUN организация_NOUN],
  [2. разг. Строгое следование своим собственным правилам; самодисциплина.,
  строгий следование собственный правило самодисциплина
  строгий_ADJ следование_NOUN собственный_ADJ правило_NOUN самодисциплина_NOUN]],
 'II ж.': [[1. Самостоятельная область научных знаний.,
  самостоятельный область научный знание
  самостоятельный_ADJ область_NOUN научный_ADJ знание_NOUN],
  [2. Учебный предмет.,
  учебный предмет
  учебный_ADJ предмет_NOUN]]}

In [57]:
# небольшая проверка
instance_to_check = dict_with_definition_instances['лавка']['I ж.'][1]
instance_to_check

[2. Скамья в саду, на улице.,
скамья сад улица
скамья_NOUN сад_NOUN улица_NOUN]

In [58]:
print(instance_to_check.raw_text)
print(instance_to_check.normalized)
print(instance_to_check.normalized_with_pos)

2. Скамья в саду, на улице.
скамья сад улица
скамья_NOUN сад_NOUN улица_NOUN


In [242]:
instance_to_check.vec[0][:10]

array([-0.48843542,  0.13313213,  0.3064579 , -0.34219706,  0.02405958,
        0.0032426 , -0.149562  , -0.07421226, -0.10938195, -0.15912783],
      dtype=float32)

In [60]:
instance_to_check.vec.shape

(1, 300)

In [61]:
v1 = model_skipgram["скамья_NOUN"].reshape(1,300)
v2 = model_skipgram["сад_NOUN"].reshape(1,300)
v3 = model_skipgram["улица_NOUN"].reshape(1,300)

In [62]:
print(v1[0][:5])
print(v2[0][:5])
print(v3[0][:5])

[-0.5760414   0.25106105  0.28208125 -0.137021    0.14280792]
[-0.23973095 -0.26112202  0.41514    -0.2574537  -0.06089966]
[-0.649534    0.40945736  0.22215244 -0.6321165  -0.0097295 ]


In [63]:
print(instance_to_check.vec[0][:5])

[-0.48843542  0.13313213  0.3064579  -0.34219706  0.02405958]


In [263]:
def all_definitions_to_excel(dict_with_definition_instances):
    """
    Сохраним в удобочитаемом виде.
    """
    c = ["lemma", "sense_name", "definition"]
    result_df = pandas.DataFrame(columns=c) # пустой датафрейм для результатов
    for lemma in dict_with_definition_instances:
        for sense_name in dict_with_definition_instances[lemma]:
            for definition in dict_with_definition_instances[lemma][sense_name]:
                list_to_append = [lemma, sense_name, definition.raw_text]
                result_df = result_df.append({'lemma':list_to_append[0], 'sense_name':list_to_append[1], 
                                              'definition':list_to_append[2]}, ignore_index=True)
    result_df.to_excel("C:\\Users\\boss\\Documents\\Diploma\\all_definitions.xlsx", index=False)

In [247]:
all_definitions_to_excel(dict_with_definition_instances)

## Нахождение самых подходящих определений 

In [112]:
class Candidate(object):
    """
    Класс для определений, которые забираем из словаря - содержит лемму, номер (или другой референс) значения и 
    инстанс специального класса для определения из словаря.
    """
    def __init__(self, lemma, sense_num, definition):
        self.lemma = lemma
        self.sense_num = sense_num
        self.definition = definition
    
    def __repr__(self):
        return "%s :: %s :: '%s'" % (self.lemma, self.sense_num, self.definition.raw_text)

In [233]:
def get_best_definitions(dict_with_sorted_good_words, dict_with_definition_instances, output_excel, 
                         look_at_lemmas_not_words=True, metric='cosine'):
    """
    Записывает в файл лучшие определения для каждого набора ключевых слов, характеризуюшего значение.
    Аргументы:
    dict_with_sorted_good_words -     словарь с отсортированными по косинусу ключевыми словами;
    dict_with_definition_instances -  словарь, содержащий определения из словаря;
    output_excel -                    excel файл, куда записывать результат;
    look_at_lemmas_not_words -        параметр, при котором мы в словаре ищем вхождения по всем возможным леммам для 
                                      переданного слова, а не по слову как есть;
    metric='cosine'/'words_overlap' - каким способом находить лучшие определения из словаря: по косинусу между средними 
                                      w2v векторами для определения и ключевых слов, либо по пересечению ключевых слов и 
                                      слов в определении.
    """
    c = ["target_word", "sense#", "key_words", "lemma", "sense_name", "definition"]
    result_df = pandas.DataFrame(columns=c) # пустой датафрейм для результатов
    
    for target_word_pos in dict_with_sorted_good_words:
        word = target_word_pos[:target_word_pos.index("_")]
        #print(word)

        if look_at_lemmas_not_words: # смотреть на слово как есть / на все его возможные леммы
            variants = morph.parse(word)
            set_of_lemmas = set([el.normal_form for el in variants]) # все возможные леммы для слова
        else:
            set_of_lemmas = {word}

        list_with_candidates = [] # соберём сюда все возможные определения
        for lemma in set_of_lemmas:
            try:
                inner_dict = dict_with_definition_instances[lemma]
            except KeyError:
                print("Леммы '%s' нет в словаре" % lemma)
                continue
            for sense_num in inner_dict:
                for definition_instance in inner_dict[sense_num]:
                    list_with_candidates.append(Candidate(lemma, sense_num, definition_instance))
        # теперь в list_with_candidates содержатся все значения (определения из словаря), которые могут относиться к
        # входному слову. А для значений вектора лежат в element.definition.vec

        for sense_num in dict_with_sorted_good_words[target_word_pos]:
            
            # если ключевых слов для данного значения нет - эксплицитно это запишем в датафрейм
            if dict_with_sorted_good_words[target_word_pos][sense_num] == []:
                list_to_append = [word, str(sense_num), "NO KEY WORDS", "-", "-", "-"]
                result_df = result_df.append({'target_word':list_to_append[0], 'sense#':list_to_append[1], 
                                              'key_words':list_to_append[2], 'lemma':list_to_append[3], 
                                              'sense_name':list_to_append[4], 'definition':list_to_append[5]}, 
                                             ignore_index=True)
                continue # и переходим к следующему
            
            if metric == 'cosine':
                array = [instance.vec for instance in dict_with_sorted_good_words[target_word_pos][sense_num]]
                mean_vec = np.mean(array, axis=0) # средний вектор для ключевых слов по имеющимся векторам для каждого слова

                cos_list = [] # запишем все значения косинусов
                for candidate in list_with_candidates:
                    c = cos(mean_vec, candidate.definition.vec).tolist()[0][0]
                    cos_list.append(c)

                # выберем определение с наибольшим значением косинуса
                max_cos = max(cos_list)
                # их теоретически мб несколько, нужно это учесть:
                list_with_best_definitions = []
                for i, value in enumerate(cos_list):
                    if value == max_cos:
                        list_with_best_definitions.append(list_with_candidates[i])

            elif metric == 'words_overlap':
                key_words = set([instance.lemma for instance in dict_with_sorted_good_words[target_word_pos][sense_num]])

                overlap_list = [] # для каждого определения запишем количество пересекающихся слов
                for candidate in list_with_candidates:
                    definition_words = set(candidate.definition.normalized.split())
                    overlap_list.append(len(key_words & definition_words)) # пересекаем множества

                # выберем определения с наибольшим пересечением
                max_overlap = max(overlap_list)
                list_with_best_definitions = []
                for i, value in enumerate(overlap_list):
                    if value == max_overlap:
                        list_with_best_definitions.append(list_with_candidates[i])

            else:
                raise ValueError("Проверьте название метрики: 'cosine' или 'words_overlap'.")

            # теперь для конкретного sense_num из dict_with_sorted_good_words[target_word_pos] мы нашли лучшее/ие определение
            # Они хранятся в list_with_best_definitions. Записываем новую строку в датафрейм
            for best in list_with_best_definitions:
                key_words = " ".join([instance.lemma for instance in dict_with_sorted_good_words[target_word_pos][sense_num]])
                list_to_append = [word, str(sense_num), key_words, best.lemma, best.sense_num, best.definition.raw_text]
                result_df = result_df.append({'target_word':list_to_append[0], 'sense#':list_to_append[1], 
                                              'key_words':list_to_append[2], 'lemma':list_to_append[3], 
                                              'sense_name':list_to_append[4], 'definition':list_to_append[5]}, 
                                             ignore_index=True)
            
    result_df.to_excel(output_excel, index=False)
    return

In [147]:
path = "C:\\Users\\boss\\Documents\\Diploma\\results\\"

In [236]:
get_best_definitions(sorted_good_words_from_perfect, dict_with_definition_instances, path+"0_errors+cosine.xlsx")
get_best_definitions(sorted_good_words_from_10_errors, dict_with_definition_instances, path+"10_errors+cosine.xlsx")
get_best_definitions(sorted_good_words_from_20_errors, dict_with_definition_instances, path+"20_errors+cosine.xlsx")
get_best_definitions(sorted_good_words_from_30_errors, dict_with_definition_instances, path+"30_errors+cosine.xlsx")
get_best_definitions(sorted_good_words_from_40_errors, dict_with_definition_instances, path+"40_errors+cosine.xlsx")

In [235]:
get_best_definitions(sorted_good_words_from_perfect, dict_with_definition_instances, path+"0_errors+overlap.xlsx", 
                     metric="words_overlap")
get_best_definitions(sorted_good_words_from_10_errors, dict_with_definition_instances, path+"10_errors+overlap.xlsx", 
                     metric="words_overlap")
get_best_definitions(sorted_good_words_from_20_errors, dict_with_definition_instances, path+"20_errors+overlap.xlsx", 
                     metric="words_overlap")
get_best_definitions(sorted_good_words_from_30_errors, dict_with_definition_instances, path+"30_errors+overlap.xlsx", 
                     metric="words_overlap")
get_best_definitions(sorted_good_words_from_40_errors, dict_with_definition_instances, path+"40_errors+overlap.xlsx", 
                     metric="words_overlap")

In [265]:
get_best_definitions(sorted_ten_good_words_from_perfect, dict_with_definition_instances, 
                     path+"0_errors_ten_words+cosine.xlsx")
get_best_definitions(sorted_ten_good_words_from_10_errors, dict_with_definition_instances, 
                     path+"10_errors_ten_words+cosine.xlsx")
get_best_definitions(sorted_ten_good_words_from_20_errors, dict_with_definition_instances, 
                     path+"20_errors_ten_words+cosine.xlsx")
get_best_definitions(sorted_ten_good_words_from_30_errors, dict_with_definition_instances, 
                     path+"30_errors_ten_words+cosine.xlsx")
get_best_definitions(sorted_ten_good_words_from_40_errors, dict_with_definition_instances, 
                     path+"40_errors_ten_words+cosine.xlsx")

In [272]:
get_best_definitions(sorted_thirty_good_words_from_perfect, dict_with_definition_instances, 
                     path+"0_errors_thirty_words+cosine.xlsx")
get_best_definitions(sorted_thirty_good_words_from_10_errors, dict_with_definition_instances, 
                     path+"10_errors_thirty_words+cosine.xlsx")
get_best_definitions(sorted_thirty_good_words_from_20_errors, dict_with_definition_instances, 
                     path+"20_errors_thirty_words+cosine.xlsx")
get_best_definitions(sorted_thirty_good_words_from_30_errors, dict_with_definition_instances, 
                     path+"30_errors_thirty_words+cosine.xlsx")
get_best_definitions(sorted_thirty_good_words_from_40_errors, dict_with_definition_instances, 
                     path+"40_errors_thirty_words+cosine.xlsx")

In [273]:
# выводим в удобочитаемом виде целевые слова, их значения и ключевые слова
for target_word in sorted_thirty_good_words_from_perfect:
    print(target_word)
    for sense_num in sorted_thirty_good_words_from_perfect[target_word]:
        list_of_kws = [good_word.lemma for good_word in sorted_thirty_good_words_from_perfect[target_word][sense_num]]
        print("%s :: [%s]" % (sense_num, " ".join(list_of_kws)))

лавка_NOUN
1 :: [изба скамейка стол дверь пол стена деревянный угол сидеть нога]
2 :: [магазин купец торговать дом улица книжный хозяин казённый сувенирный мясной церковь торговый ряд книга деньга город]
замок_NOUN
1 :: [крепость замковый башня стена деревянный король князь]
2 :: [замковый засов ворот ключ дверь храм навесный]
винт_NOUN
1 :: [болт гайка пружина регулировочный ручка крепление отверстие крышка крепиться головка панель регулировка диаметр резьба конструкция деталь]
2 :: [лопасть мотор двигатель несущий вращение гребной вращаться диаметр самолёт вертолёт корабль лодка машина скорость вода мощность воздух полёт]
лук_NOUN
1 :: [стрела тетива стрельба лучник луковый натянуть оружие]
2 :: [чеснок луковица гусиный сок блюдо овощной растение ветвистый]
суда_NOUN
1 :: [арбитражный судно апелляционный судья судебный конституционный дело решение верховный президиум]
2 :: [судно корабль флот]
дисциплина_NOUN
1 :: [строгий нарушение вера учебный]
2 :: [преподавание обучение наука уче